In [3]:
import numpy as np
import matplotlib.pylab as plt
import math
import pandas as pd
import seaborn as sns
sns.set()
from matplotlib import rcParams
rcParams['figure.figsize'] = 11.7,8.27

In [17]:
class RLS_2:
    def __init__(self, num_vars, lam, delta):
        '''
        num_vars: Degree of polinomial
        lam: Forgetting factor, usually very close to 1
        delta: Initation value -> ! Bad initation needs more itteration to reach same accuracy
        '''
        self.num_vars = num_vars
        self.P = delta*np.matrix(np.identity(self.num_vars))

        # Weigths/Coefficent of the system
        self.w = np.matrix(np.zeros(self.num_vars))
        self.w = self.w.reshape(self.w.shape[1],1)

        # Kalman Gain Factor
        self.g = np.matrix(np.zeros(num_vars))
        self.g = np.reshape(self.g,(num_vars,1))
        
        # Variables needed for add_obs
        self.lam_inv = lam**(-1)
        
        # A priori error
        self.a_priori_error = 0
        
        # Count of number of observations added
        self.num_obs = 0

    def add_obs(self, x, t):
        '''
        Expected value is t, add the new observation as t.
        t is noisy output of the some linear system. Input of the RLS.

        Task is to identify a system, by determining coefficents,
        that outputs a value which is closest to t.

        x is a column vector as a numpy matrix  |   (new inputs to the system)
        t is a real scalar                      |   (expected output to update weigths)
        '''            

        self.g = self.lam_inv*self.P*x/(1+self.lam_inv*(x.T*self.P*x))
        self.P = self.P*self.lam_inv - self.g*(x.T*self.P)
        self.w = self.w + self.g*(t-x.T*self.w)
        self.num_obs += 1

In [61]:
test_size = 1000
# Test function
f = lambda x: 0.07*x**4-1.9*x**3-7.9*x**2+13*x
y = np.array([f(i) for i in range(test_size)])
noise = np.random.normal(loc = 7, scale = 2 ,size = test_size)
noisy_y = y + noise

num_vars = 5
lam = 0.98
LS_2 = RLS_2(num_vars,lam,1)

In [71]:
pred_x = []
pred_y = []

for i in range(500):
    x = np.matrix(np.zeros((1,num_vars)))
    for j in range(num_vars):
        x[0,j] = i**j 
    pred_x.append(i)
    pred_y.append(float(x*LS_2.w))
    LS_2.add_obs(x.T,noisy_y[i])

    if(i % 25 == 0):
        print("Weigths at trail [{}]:".format(LS_2.num_obs -1), LS_2.w.T)

Weigths at trail [4500]: [[ 3.31476855 13.04051374 -7.90016003 -1.89999973  0.07      ]]
Weigths at trail [4525]: [[ 6.58757825 12.98246672 -7.89984812 -1.9000004   0.07      ]]
Weigths at trail [4550]: [[ 6.19160918 13.01773333 -7.90008806 -1.89999984  0.07      ]]
Weigths at trail [4575]: [[ 5.94218956 13.03853958 -7.90028022 -1.89999928  0.07      ]]
Weigths at trail [4600]: [[ 6.06764407 13.04176314 -7.90045389 -1.89999855  0.07      ]]
Weigths at trail [4625]: [[ 6.12835703 13.03866831 -7.90041958 -1.89999866  0.07      ]]
Weigths at trail [4650]: [[ 6.32985443 13.02868699 -7.9003106  -1.899999    0.07      ]]
Weigths at trail [4675]: [[ 6.69364267 13.01348334 -7.9001626  -1.89999944  0.07      ]]
Weigths at trail [4700]: [[ 7.20337813 12.99639249 -7.90002887 -1.89999973  0.07      ]]
Weigths at trail [4725]: [[ 7.73769264 12.97721952 -7.89986525 -1.90000017  0.07      ]]
Weigths at trail [4750]: [[ 7.72806129 12.97109021 -7.89974586 -1.90000078  0.07      ]]
Weigths at trail [477